In [3]:
from os import listdir, getcwd
from os.path import join
import os
from datetime import datetime as dt
import sqlite3 as sql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import constants as c

In [4]:
conn = sql.connect(c.DATABASE)
cursor = conn.cursor()

In [5]:
def prepare_database():
    view = """
        create view lines_of_code as
        select distinct blocks_mfiles.id_block, code
        from lines_mfiles, lines_tokens, blocks_mfiles
        where lines_mfiles.id_line = lines_tokens.id_line
        and lines_tokens.id_block = blocks_mfiles.id_block
        order by lines_mfiles.id_mfile, lines_tokens.id_block, lines_mfiles.line_nbr
        """
    cursor.execute(view)
    
    index = "create table lines_of_code_table as select * from lines_of_code"
    cursor.execute(index)

    index = "create index lines_of_code_table_id_block on lines_of_code_table (id_block)"
    cursor.execute(index)


def code_by_block(files: int):
    query = """select *
    from lines_of_code_table
    where lines_of_code_table.id_block <""" + str(files)
    cursor.execute(query)
    with open('results.csv', 'w') as results:
        print('block,code', file=results)
        for row in cursor:
            print(f'{row[0]},{row[1]}', file=results)


def change_configs():
    with open(database_configs_location, "w") as database_configs:
        print("False", file=database_configs)


In [6]:
if False:
    prepare_database() # Correr sempre que mudar a base de dados
    change_configs()

number_of_files = input("How many blocks of code?")
code_by_block(number_of_files)

In [ ]:
with open('results.csv') as results:
    for entry in results.readlines()[1:]:
        index = entry.index(',')
        line = entry[:index]
        code = entry[index+1:-1]
        with open(f'{c.MATLAB_FILES}\\{line}.m', 'a') as create_file:
            create_file.write(code)
            create_file.write('\n')